In [7]:
library(tidyverse)
library(repr)
library(rvest)
library(stringr)
library(readxl)
library(dplyr)
options(repr.matrix.max.rows = 6)

In [8]:
data <- read_csv("https://raw.githubusercontent.com/thisisyifulin/group_project-group34-/main/Training_Data_User_Modeling_Dataset_Hamdi%20Tolga%20KAHRAMAN.csv") |>
select(STG, SCG, STR, LPR, PEG, UNS)

new_data <- data |>
select(STG, STR, LPR, PEG) # selecting the variables

A_grade <- filter(new_data, 1.00 > PEG, PEG > 0.80) |> # changing PEG from decimals to letter grades
mutate(PEG = "A")

B_grade <- filter(new_data, 0.79 > PEG, PEG > 0.65) |> 
mutate(PEG = "B")

C_grade <- filter(new_data, 0.64 > PEG, PEG > 0.55) |> 
mutate(PEG = "C")

D_grade <- filter(new_data, 0.54 > PEG, PEG > 0.50) |> 
mutate(PEG = "D")

F_grade <- filter(new_data, 0.49 > PEG, PEG > 0.00) |> 
mutate(PEG = "F")

mutated_data <- bind_rows(A_grade, B_grade, C_grade, D_grade, F_grade)

mutated_data

ERROR: Error in open.connection(structure(4L, class = c("curl", "connection"), conn_id = <pointer: 0x28c>), : HTTP error 404.
